In [ ]:
import geopandas as gp
import pandas as pd
import contextily as cx
import osmnx as ox
import matplotlib.pyplot as plt

In [ ]:
importNational = pd.read_csv('Data/EpaFlightNational.csv')
importNational = importNational.reset_index()
for index, row in importNational.iterrows():
    if ((row['STATE'] == 'VI') or (row['STATE'] == 'GU') or (row['STATE'] == 'PR') or (row['STATE'] == 'AS') or (row['STATE'] == 'MP')):
        importNational = importNational.drop(index)
nationalEmissions = gp.GeoDataFrame(
    importNational, geometry=gp.points_from_xy(importNational.LONGITUDE, importNational.LATITUDE), crs = 'EPSG:4326'
)
nationalEmissions.plot(column="GHG QUANTITY (METRIC TONS CO2e)", legend=True, cmap = "autumn")

In [ ]:
nationalPowerLines = gp.read_file('Data/Electric_Power_Transmission_Lines.geojson')
nationalLinesBuffers = nationalPowerLines.buffer(0.043, resolution=16, cap_style='round', join_style='round', mitre_limit=5.0, single_sided=False)
nationalLinesBuffers.to_crs('EPSG:4326')
nationalBuffersGDF = gp.GeoDataFrame(nationalLinesBuffers, geometry=gp.GeoSeries(nationalLinesBuffers))
nationalBuffersGDF.plot()

In [ ]:
overlayNational = nationalBuffersGDF.overlay(nationalEmissions, how="intersection", keep_geom_type =False)

nationalInGridSources = pd.DataFrame(overlayNational.drop(columns=['geometry', 0]))
nationalInGridSources = nationalInGridSources.drop_duplicates()

stateList = [ 'AL', 'AK', 'AS', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FM', 'FL', 'GA', 'GU', 'HI', 'ID', 'IL', 'IN', 'IA', 
             'KS', 'KY', 'LA', 'ME', 'MH', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 
             'ND', 'MP', 'OH', 'OK', 'OR', 'PW', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VI', 'VA', 'WA', 'WV', 'WI', 'WY' ]

for i in stateList : 
    stateInGridSources = nationalInGridSources.query('STATE' == stateList)
    stateSources = importNational.query('STATE' == stateList)

    totalStateEmissions = stateSources['GHG QUANTITY (METRIC TONS CO2e)'].sum()
    stateEmissionsGrid = stateInGridSources['GHG QUANTITY (METRIC TONS CO2e)'].sum()
    percentageEmissions = stateEmissionsGrid / totalStateEmissions
    
    print(i)
    print(percentageEmissions)
    print("\n")